In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import csv, time, random, os

# --- CONFIGURATION ---
chrome_options = Options()
chrome_options.add_argument("--headless")  # run browser in background
chrome_options.add_argument("--window-size=1920,1080")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("start-maximized")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36")

driver = webdriver.Chrome(options=chrome_options)  # make sure chromedriver is installed

# --- HELPER FUNCTION TO CLEAN AMAZON URL ---
def clean_amazon_url(url):
    if "/dp/" in url:
        part = url.split("/dp/")[1]
        asin = part.split("/")[0].split("?")[0]
        return f"https://www.amazon.in/dp/{asin}"
    return url

# --- FUNCTION TO SCRAPE PRODUCT ---
def scrape_product(url):
    try:
        driver.get(url)
        time.sleep(random.uniform(2,4))  # wait for page to load

        # TITLE
        try:
            title = driver.find_element(By.ID, "productTitle").text.strip()
        except:
             title = "NA"

        # PRICE
        try:
           price = driver.find_element(By.ID, "priceblock_ourprice").text
        except:
           try:
              price = driver.find_element(By.ID, "priceblock_dealprice").text
           except:
        # Fallback for price inside nested spans
             try:
               price_whole = driver.find_element(By.CSS_SELECTOR, "span.a-price-whole").text
               price_symbol = driver.find_element(By.CSS_SELECTOR, "span.a-price-symbol").text
               price = price_symbol + price_whole
             except:
               price = "NA"

        # RATING
        try:
            rating = driver.find_element(By.CSS_SELECTOR, "span.a-icon-alt").get_attribute('textContent').strip()
        except:
            try:
                rating = driver.find_element(By.CSS_SELECTOR, "span[data-hook='rating-out-of-text']").get_attribute('textContent').strip()
            except:
                rating = "NA"
        # REVIEW COUNT
        try:
            reviews = driver.find_element(By.ID, "acrCustomerReviewText").text.strip()
        except:
            reviews = "NA"

        # AVAILABILITY
        try:
            avail = driver.find_element(By.ID, "availability").text.strip()
        except:
            avail = "NA"

        return [title, price, rating, reviews, avail]

    except Exception as e:
        print("Error scraping:", e)
        return ["NA"]*5

# --- MAIN ---
if __name__ == "__main__":
    input_file = r"C:\Users\Dell\Downloads\url.txt"
    output_file = os.path.join(os.getcwd(), "out.csv")
    print("Saving CSV to:", output_file)

    with open(input_file, "r") as infile, open(output_file, "w", newline="", encoding="utf-8") as outfile:
        writer = csv.writer(outfile)
        writer.writerow(["Title", "Price", "Rating", "Reviews", "Availability"])

        for link in infile:
            if not link.strip():
                continue
            url = clean_amazon_url(link.strip())
            print(f"Scraping: {url}")
            data = scrape_product(url)
            print("→", data)
            writer.writerow(data)
            time.sleep(random.uniform(2,5))  # avoid being blocked

    driver.quit()


Saving CSV to: C:\Users\Dell\out.csv
Scraping: https://www.amazon.in/gp/aw/d/B0CZXZG1BW/?_encoding=UTF8&pd_rd_plhdr=t&aaxitk=b6356f8d1debd3963fe3c682494c2e53&hsa_cr_id=0&sr=1-2-e0fa1fdd-d857-4087-adda-5bd576b25987&aref=qoN0CrtOZA&ref_=sbx_be_s_sparkle_dlcd_asin_1_img
→ ['ZAVERI PEARLS Green Meenakari Kundan Bridal Long Dazzling Necklace Earring Maangtikka Set For Women-ZPFK17761', '₹999', '4.1 out of 5 stars', '299 ratings', 'In stock']
Scraping: https://www.amazon.in/dp/B0CSD8NHXW
→ ['SALTY Anti Tarnish Fancy Clover Gift Box | Necklace, Earrings & Bracelet Jewellery Set for Women | Latest Stylish Minimal Jewelry Sets | Modern Design Aesthetic Accessories | Ideal for Gifting', '₹449', '4.0 out of 5 stars', '95 ratings', 'In stock']
Scraping: https://www.amazon.in/dp/B0BQL21KRP
→ ['ZENEME Rhodium-Plated with Silver-Toned Floral Design White American Diamond Studded Jewellery Set', '₹909', '4.3 out of 5 stars', '186 ratings', 'In stock']
Scraping: https://www.amazon.in/dp/B0B15GRS3X
→ ['